In [96]:
import requests
from bs4 import BeautifulSoup
import html
import csv
import re
import concurrent.futures

# Make a request to the website
mainurl = "https://www.alodokter.com"
url = "https://www.alodokter.com/komunitas/diskusi/penyakit/page/2"
response = requests.get(url)
response.encoding = 'utf-8'
csv_file = 'output.csv'

soup = BeautifulSoup(response.content, 'html.parser')

# Find all card-topic elements
href_links = [link.get('href') for link in soup.find_all('card-topic')]

def scrape_content(link):
    child_url = mainurl + link
    responses = requests.get(child_url)
    responses.encoding = 'utf-8'
    souper = BeautifulSoup(responses.content, 'html.parser')
    
    title = souper.find('detail-topic').get('member-topic-content')
    content = souper.find('doctor-topic').get('doctor-topic-content')
    
    return title[1:-1], content[1:-1]

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(scrape_content, href_links)

all_title, all_content = zip(*results)

In [99]:

def html_decode(text):
    teks = text.replace('\\n', '\n').replace('\\u003c', '\u003c').replace('\\u003e', '\u003e')
    print(teks)
    teks = re.sub(r"<[^>]+>", "", teks)
    return teks
    
all_content = [html_decode(content) for content in all_content]
all_title = [html_decode(title) for title in all_title]

print(all_content[0])




Alo, selamat siang
Keluhan yang anda alami merupakan sakit tenggorokan, dimana sakit tenggorokan ialah suatu kondisi rasa nyeri, gatal, tidak nyaman dan kering di tenggorokan yang domana kondisi ini merupakan suatu gejala yang bisa timbul akibat beragam gangguan atau penyakit. Sakit tenggorokan yang anda alami ini bisa disebabkan oleh beberapa hal seperti :

Infeksi virus akibat flu atau covid
Infeki bakteri akibat radang tenggorokan
Penyakit asam lambung yang dapat mengiritasi tenggorokan akibat asam lambung yang naik keatas kerongkongan

Untuk penanganan sakit tenggorokan anda alami ada beberapa tips yang dapat anda lakukan untuk mengurangi atau meredai keluhan seperti :

Konsumsi air putih yang banyak minimal 2 liter dalam sehari
Melakukan kumur-kumur dengan air garam hangat atau menggunakan obat kumur antiseptik
Konsumsi obat pereda nyeri seperti paracetamol per 8 jam
Konsumsi obat isap mengandung antiseptik untuk tenggorokan yang dapat anda beli bebas di apotik
Konsumsi makanan 

In [98]:
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    for content in all_content:
      writer.writerow([content])